In [2]:
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd
import sys
import scipy
import matplotlib
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.svm import SVC
# compare algorithms
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.utils import resample
import seaborn as sns
from numpy import loadtxt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer 
import matplotlib.pyplot as plt
from numpy import loadtxt
from sklearn.ensemble import RandomForestClassifier

dfold=pd.read_csv('online_jewellery_shop_train.csv')

In [4]:
#Converting Month
def month_converter(month):
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'June', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    return months.index(month) + 1

In [5]:
df = dfold.dropna()
df['VisitorType'] = df['VisitorType'].map({'Returning_Visitor': 1, 'New_Visitor': 0,'Other': 2})
df['Month'] = df['Month'].apply(month_converter)
df['Administrative_Duration'] = np.log(df['Administrative_Duration'].replace(0, np.nan)).fillna(0)
df['Informational_Duration'] = np.log(df['Informational_Duration'].replace(0, np.nan)).fillna(0)
df['Administrative'] = np.log(df['Administrative'].replace(0, np.nan)).fillna(0)
df['ProductRelated_Duration'] = np.log(df['ProductRelated_Duration'].replace(0, np.nan)).fillna(0)
df['ProductRelated'] = np.log(df['ProductRelated'].replace(0, np.nan)).fillna(0)
df['Informational'] = np.log(df['Informational'].replace(0, np.nan)).fillna(0)



#df = df[((df.Informational_Duration - df.Informational_Duration.mean()) / df.Informational_Duration.std()).abs() < 3]
#df = df[((df.Administrative_Duration - df.Administrative_Duration.mean()) / df.Administrative_Duration.std()).abs() < 3]
#df = df[((df.PageValues - df.PageValues.mean()) / df.PageValues.std()).abs() < 3]
#df = df[((df.Browser - df.Browser.mean()) / df.Browser.std()).abs() < 3]
#df = df[((df.ProductRelated_Duration - df.ProductRelated_Duration.mean()) / df.ProductRelated_Duration.std()).abs() < 3]

/Users/Betty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/Betty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/Betty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in log
  after removing the cwd from sys.path.
/Users/Betty/anaconda3/lib/python3.7/site-packages/ipyke

In [6]:
#resample the data
df_majority = df[df.Revenue==1]
df_minority = df[df.Revenue==0]
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=4450,    # to match majority class 30/70 
                                 random_state=123) # reproducible results
df_upsampled = pd.concat([df_majority, df_minority_upsampled])
df_upsampled.Revenue.value_counts()

0    4450
1    1335
Name: Revenue, dtype: int64

In [7]:
X = df_upsampled[df_upsampled.columns[:-1]]
X = X[X.columns[1:]]
y = df_upsampled.iloc[:,-1]

In [98]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
random_grid

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000],
 'max_features': ['auto', 'sqrt'],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'min_samples_split': [2, 5, 10],
 'min_samples_leaf': [1, 2, 4],
 'bootstrap': [True, False]}

In [99]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X, y)
rf_random.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 14.8min finished
/Users/Betty/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'n_estimators': 400,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 30,
 'bootstrap': True}

In [8]:
from sklearn.feature_selection import SelectFromModel
model = SelectFromModel(RandomForestClassifier(n_estimators=400, max_depth= 30, random_state=42, 
                               min_samples_leaf=1, max_features='sqrt',min_samples_split = 5 ))
model.fit(X, y)
selected_feat= X.columns[(model.get_support())]
selected_feat

Index(['ProductRelated', 'ProductRelated_Duration', 'BounceRates', 'ExitRates',
       'PageValues'],
      dtype='object')

In [9]:
finaltest = pd.read_csv('online_jewellery_shop_test_final.csv')
finaltest = finaltest.fillna(0)
finaltest['Month'] = finaltest['Month'].apply(month_converter)
finaltest['VisitorType'] = finaltest['VisitorType'].map({'Returning_Visitor': 1, 'New_Visitor': 0,'Other': 2})
finaltest['Administrative_Duration'] = np.log(finaltest['Administrative_Duration'].replace(0, np.nan)).fillna(0)
finaltest['Informational_Duration'] = np.log(finaltest['Informational_Duration'].replace(0, np.nan)).fillna(0)
finaltest['Administrative'] = np.log(finaltest['Administrative'].replace(0, np.nan)).fillna(0)
finaltest['ProductRelated_Duration'] = np.log(finaltest['ProductRelated_Duration'].replace(0, np.nan)).fillna(0)
finaltest['ProductRelated'] = np.log(finaltest['ProductRelated'].replace(0, np.nan)).fillna(0)
newX = finaltest[finaltest.columns[1:]]
newy = model.predict_proba(newX)

/Users/Betty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in log
  """
/Users/Betty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in log
  
/Users/Betty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log
  


AttributeError: 'SelectFromModel' object has no attribute 'predict_proba'

In [10]:
ID = finaltest.iloc[:,0]
pred_prob = newy[:,1]
dffinal = pd.DataFrame({'ID':ID, 'Revenue':pred_prob})

NameError: name 'newy' is not defined

In [104]:
dffinal.to_csv(r'Final_prediction.csv', index = False)